In [10]:
# https://ppiazi.tistory.com/entry/python-cyclomatic-complexity-%EB%AA%A8%EB%93%88-lizard

In [ ]:
# https://github.com/terryyin/lizard

In [41]:
#customized preprocessing pkg
import preprocess as pp
import pandas as pd
import numpy as np
import pickle
import lizard
import pprint

In [42]:
# 파일 불러오기
with open('../../data/complexity_df.pkl', 'rb') as f:
    complexity_df = pickle.load(f)

In [43]:
complexity_df.head()

,id,cdate,title,body
0,74574268,2022-11-25,stateStore.delete(key) in Kafka is not working,<p>I have what it thought would be a simple st...
1,74574269,2022-11-25,How to write a nested loop over several argume...,<p>The following code</p>\n<pre><code>macro_ru...
2,74574270,2022-11-25,updating sql query skips indexes,<p>I am trying to save user search results in ...
3,74574306,2022-11-25,"Add ""collection"" of attributes directly to top...",<p>I am trying to capture (S3) logs in a struc...
4,74574308,2022-11-25,Multiple images (logo and a background image) ...,<p>I'm trying to use a logo plus a background ...


In [44]:
complexity_df['body']

0         <p>I have what it thought would be a simple st...
1         <p>The following code</p>\n<pre><code>macro_ru...
2         <p>I am trying to save user search results in ...
3         <p>I am trying to capture (S3) logs in a struc...
4         <p>I'm trying to use a logo plus a background ...
                                ...                        
229298    <p>is it possible to use SageMaker Clarify bia...
229299    <p>I have created a custom form in a custom mo...
229300    <p>I am still struggling with using data from ...
229301    <p>I am fetching names from tags in wikipedia ...
229302    <p>I want to use shared memory to transfer lar...
Name: body, Length: 229303, dtype: object

In [45]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()

In [46]:
complexity_df['c_h_body'] = complexity_df['body'].apply(lambda x : htmlp.clean_html_str(x))

In [47]:
complexity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229303 entries, 0 to 229302
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        229303 non-null  int64 
 1   cdate     229303 non-null  object
 2   title     229303 non-null  object
 3   body      229303 non-null  object
 4   c_h_body  229303 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.7+ MB


In [48]:
complexity_df['c_body'] = complexity_df['body'].apply(lambda x : codep(x))

In [49]:
complexity_df['c_body'].iloc[0:3].apply(lambda x : x['code_sections'][0]['span_str'])

0    @Override\npublic void punctuate(long l) {\n  ...
1    macro_rules! test {\n    ( $( $x1:expr ),*; bl...
2    $sql = &quot;INSERT INTO search_data (SEARCH, ...
Name: c_body, dtype: object

In [62]:
complexity_df.loc[0, 'c_body']['code_sections'][0]['span_str']

'@Override\npublic void punctuate(long l) {\n    log.info(&quot;PeriodicRetryPunctuator started: &quot; + l);\n\n    try(KeyValueIterator&lt;String, TestEventObject&gt; iter = stateStore.all()) {\n        while(iter.hasNext()) {\n            KeyValue&lt;String, TestEventObject&gt; keyValue = iter.next();\n            String key = keyValue.key;\n            TestEventObject event = keyValue.value;\n            \n            try {\n                log.info(&quot;Event: &quot; + event);\n                // Sends event over HTTP. Will throw HttpResponseException if 404 is received\n                eventService.processEvent(event);\n                \n                stateStore.delete(key);\n                stateStore.flush();\n                \n                // Check that statestore returns null\n                log.info(&quot;Check: &quot; + stateStore.get(key));\n            } catch (HttpResponseException hre) {\n                log.info(&quot;Periodic retry received 404. Retrying at nex

In [58]:
complexity_df.loc[0, 'c_body'].apply(lambda x : x['code_sections'][0]['span_str'])

AttributeError: 'dict' object has no attribute 'apply'

In [ ]:
complexity_df['c_body'].iloc[0:3].apply(lambda x : x['code_sections'][0]['span_str'])

In [50]:
# 전체 데이터셋 
for x in range(3):
    with open('../../data/complexity/'+str(x)+'.java', 'wb') as f:
        pickle.dump(complexity_df['c_body'].iloc[0:3].apply(lambda x : x['code_sections'][0]['span_str'])[x]
                    , f)


In [53]:
i = lizard.analyze_file("../../data/complexity/a.java")

In [54]:
pprint.pp(i.__dict__)

{'filename': '../../data/complexity/a.java',
 'nloc': 24,
 'function_list': [<lizard.FunctionInfo object at 0x173676e50>],
 'token_count': 218}


In [55]:
pprint.pp(i.function_list[0].__dict__)

{'cyclomatic_complexity': 4,
 'nloc': 23,
 'token_count': 214,
 'name': 'punctuate',
 'long_name': 'punctuate( long l)',
 'start_line': 2,
 'end_line': 30,
 'full_parameters': ['long l'],
 'filename': '../../data/complexity/a.java',
 'top_nesting_level': 0,
 'fan_in': 0,
 'fan_out': 0,
 'general_fan_out': 0}
